In [1]:
!ls /workspace/llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-Instruct-150K/translated/

llava_instruct_150k.json  llava_instruct_80k.json


In [5]:
!head -n 30 /workspace/llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-Instruct-150K/translated/llava_instruct_150k.json

# llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-Instruct-150K/translated/llava_instruct_150k.json  
# llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-Instruct-150K/translated/llava_instruct_80k.json

[
  {
    "id": "000000033471",
    "image": "000000033471.jpg",
    "conversations": [
      {
        "from": "human",
        "value": "<image>\n图中巴士的颜色是 什么?"
      },
      {
        "from": "gpt",
        "value": "图中的公共汽车是白色和红色。"
      },
      {
        "from": "human",
        "value": "公共汽车后部可看到什么特征?"
      },
      {
        "from": "gpt",
        "value": "公共汽车的后部设有广告。"
      },
      {
        "from": "human",
        "value": "公共汽车在街上行走 吗?还是停靠在一边 呢?"
      },
      {
        "from": "gpt",
        "value": "公共汽车正在沿着拥挤着人和其他车辆的街道行驶。"
      }
    ]


In [7]:
!ls  /workspace/llm_datasets/Visual-Instruction-Tuning

coco  gqa  llava_v1_5_mix665k.json  ocr_vqa  text_vqa  vg


In [10]:
!pip install pickleshare


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
%cd llm_datasets

[Errno 2] No such file or directory: 'llm_datasets'
/workspace/llm_datasets


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [ ]:
# merge chinese to llava-pretrain
# merge 
# /workspace/llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-CC3M-Pretrain-595K/chat-translated.json
# to
# /workspace/llm_datasets/LLaVA-Pretrain/blip_laion_cc_sbu_558k.json

In [18]:
# backup
!cp /workspace/llm_datasets/LLaVA-Pretrain/blip_laion_cc_sbu_558k.json /workspace/llm_datasets/LLaVA-Pretrain/blip_laion_cc_sbu_558k.json.backup

In [7]:
from pathlib import Path
import json
from tqdm import tqdm

sources = [
    {
        "json": Path('/workspace/llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-CC3M-Pretrain-595K/chat-translated.json'),
        "image": Path('/workspace/llm_datasets/LLaVA-Pretrain'),
        "flag": "cc3m",
    },
    {
        "json": Path('/workspace/llm_datasets/LLaVA-Pretrain/blip_laion_cc_sbu_558k.json'),
        "image": Path('/workspace/llm_datasets/LLaVA-Pretrain'),
        "flag": "laion",
    }
    
]

target = Path('/workspace/llm_datasets/LLaVA-Pretrain/pretrain_data.json')

output = []
for source in sources:
    with source['json'].open('r', encoding='utf-8') as fp:
        data = json.load(fp)
        for row in tqdm(data):
            if source['flag'] == 'cc3m':
                row['image'] = f"images/{row['image']}"
                
            # image_file = source['image'] / row['image']
            # if not image_file.exists():
            #     print(row)
            #     continue

            output.append(row)


with target.open('w', encoding='utf-8') as fp:
    json.dump(output, fp, indent=4, ensure_ascii=False)

100%|██████████| 558128/558128 [00:00<00:00, 2453243.46it/s]


In [8]:
!du -sh /workspace/llm_datasets/LLaVA-Pretrain/pretrain_data.json

447M	/workspace/llm_datasets/LLaVA-Pretrain/pretrain_data.json


In [4]:
from pathlib import Path
import json
from tqdm import tqdm
import random

sources = [
    {
        "json": Path('/workspace/llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-Instruct-150K/translated/llava_instruct_150k.json'),
        "image": Path('/workspace/llm_datasets/Visual-Instruction-Tuning'),
        "image_prefix": "coco/train2017",
        "flag": "llava_150k",
        "sampling_rate": 0.02,
    },
    {
        "json": Path('/workspace/llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-Instruct-150K/translated/llava_instruct_80k.json'),
        "image": Path('/workspace/llm_datasets/Visual-Instruction-Tuning'),
        "image_prefix": "coco/train2017",
        "flag": "llava_80k",
        "sampling_rate": 0.04,
    },
    {
        "json": Path('/workspace/llm_datasets/tongue/tongue_train_data5.json'),
        "image": Path('/workspace/llm_datasets/Visual-Instruction-Tuning'),
        "image_prefix": "tongue_image_336",
        "flag": "tongue",
        "sampling_rate": 0.1,
    },
    # {
    #     "json": Path('/workspace/llm_datasets/Visual-Instruction-Tuning/llava_v1_5_mix665k.json'),
    #     "image": Path('/workspace/llm_datasets/Visual-Instruction-Tuning'),
    #     "image_prefix": "",
    #     "flag": "mix665k",
    # }
    
]

target = Path('/workspace/llm_datasets/Visual-Instruction-Tuning/finetune_data_chinese-5k.json')

output = []
for source in sources:
    with source['json'].open('r', encoding='utf-8') as fp:
        data = json.load(fp)
        for row in tqdm(data):
            if 'image' not in row:
                output.append(row)
                continue
            
            if source['flag'] == 'mix665k' and row['image'].startswith('coco/train2017/'):
                # 跳过 coco 数据集，使用中文翻译数据集
                continue
        
            if source['image_prefix']:
                 row['image'] = (Path(source['image_prefix']) / Path(row['image'])).as_posix()

            if 'sampling_rate' in source:
                if random.random() <= source['sampling_rate']:
                    image_file = source['image'] / row['image']
                    if not image_file.exists():
                        # print(row)
                        continue
                        
                    output.append(row)
            else:
                image_file = source['image'] / row['image']
                if not image_file.exists():
                    # print(row)
                    continue
                output.append(row)

print(len(output))
with target.open('w', encoding='utf-8') as fp:
    json.dump(output, fp, indent=4, ensure_ascii=False)

100%|██████████| 93367/93367 [00:10<00:00, 8834.01it/s] 


15701


In [27]:
!tail -n 100 /workspace/llm_datasets/Visual-Instruction-Tuning/finetune_data.json

                "from": "gpt",
                "value": "Possible classification tags for the listed problems could be:\n\n* Security Audit Management:\n\t+ Security audit checklist management\n\t+ Outdated checklist items and change tracking\n\t+ Audit scope definition\n\t+ Nessus account management\n\t+ Shadow and parallel audits arrangement\n\t+ Rescheduling issues\n* Communication and Collaboration:\n\t+ Too many team groups\n\t+ Consolidation of pre-release group chats\n\t+ Clear communication channels and roles\n\t+ Last-minute audit requests\n\t+ Making audit scope a required field\n* Training and Process Improvement:\n\t+ Newcomers training process improvement\n\nNote that some of the problems could be classified under multiple tags, and the above tags are just one possible way to categorize them."
            }
        ]
    },
    {
        "id": "WpsM80Y_0",
        "model": "",
        "conversations": [
            {
                "from": "human",
                "value"

In [29]:
!du -sh /workspace/llm_datasets/Visual-Instruction-Tuning/finetune_data_chinese.json

360M	/workspace/llm_datasets/Visual-Instruction-Tuning/finetune_data_chinese.json


In [2]:
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [36]:

import torch

# 创建两个张量
tensor1 = torch.tensor([1, 2, 3])
tensor2 = torch.tensor([4, 5, 6])

# 在维度0上堆叠两个张量
stacked_tensor = torch.stack([tensor1, tensor2], dim=1)

print(stacked_tensor)

tensor([[1, 4],
        [2, 5],
        [3, 6]])
